In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

In [ ]:
vggmodel = applications.VGG16(include_top=False, weights='imagenet')

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
generator = datagen.flow_from_directory('data/train',
                                       target_size=(150,150),
                                       batch_size=16,
                                       class_mode=None,
                                       shuffle=False)

bottleneck_features_train = vggmodel.predict_generator(generator, 2000/16)

np.save(open('bottleneck_features_train.npy','wb'), bottleneck_features_train)


In [ ]:
generator = datagen.flow_from_directory('data/validation',
                                       target_size=(150,150),
                                       batch_size=16,
                                       class_mode=None,
                                       shuffle=False)

bottleneck_features_validation = vggmodel.predict_generator(generator, 800/16)

np.save(open('bottleneck_features_validation.npy','wb'), bottleneck_features_validation)

In [ ]:
train_data = np.load(open('bottleneck_features_train.npy','rb'))

train_labels = np.array([0]*1000 + [1]*1000)

In [ ]:
validation_data = np.load(open('bottleneck_features_validation.npy','rb'))

validatin_labels = np.array([0]*400 + [1]*400)

In [ ]:
train_data.shape

In [ ]:
model = Sequential()

model.add(Flatten(input_shape=(4,4,512)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels,
         epochs=30,
         batch_size=16,
         validation_data=(validation_data, validation_labels))

In [ ]:
model.save_weights('bottleneck_fc_model.h5')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
img = plt.imread('dog.jpg')

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
img = load_img('dog.jpg', target_size=(150,150))

img = img_to_array(img) # maybe

img = np.expand_dims(img, axis=0)

In [ ]:
from scipy.misc import imresize

In [ ]:
img = imresize(img, (150,150,3))

In [ ]:
img = np.expand_dims(img, axis=0)

In [ ]:
img.shape

In [ ]:
convolved_img = vggmodel.predict(img,1)

In [ ]:
convolved_img.shape

In [ ]:
new_dn_model = Sequential()

new_dn_model.add(Flatten(input_shape=(4,4,512)))
new_dn_model.add(Dense(256, activation='relu'))
new_dn_model.add(Dropout(0.5))
new_dn_model.add(Dense(1, activation='sigmoid'))

In [ ]:
new_dn_model.load_weighs('bottleneck_fc_model.h5')

In [ ]:
new_model.predict_classes(convolved_img)

# STYLE TRANSFER

In [2]:
import time
from PIL import Image
import numpy as np

from keras import backend
from keras.models import Model
from keras import applications

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave, imresize

In [19]:
content_image = Image.open('content.jpg')
content_image = imresize(content_image, (300,300,3))
content_image.shape

(300, 300, 3)

In [20]:
style_image = Image.open('style.jpg')
style_image = imresize(style_image, (300, 300, 3))
style_image.shape

(300, 300, 3)

In [21]:
content_image = content_image.astype('float32')
style_ = style_image.astype('float32')

content_array = np.expand_dims(content_image, axis=0)
style_array = np.expand_dims(style_image, axis=0)

In [22]:
content_array.shape

(1, 300, 300, 3)

In [24]:
style_array.shape

(1, 300, 300, 3)

# FINAL STYLE TRANSFER

In [26]:
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras import applications
from keras import backend as K

In [53]:
img_width = 300
img_height = 300
content_weight = 0.025  # the weight given to content loss
style_weight = 1.0    # the weight given to style loss
tv_weight = 1.0     # the weight given to total variance loss

In [33]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_width,img_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = applications.vgg16.preprocess_input(img)
    return img

In [28]:
def deprocess_image(x):
    x = x.reshape((img_width,img_height, 3))
    
    # remove zero center by mean pixel
    x[:,:,0] += 103.939
    x[:,:,1] += 116.779
    x[:,:,2] += 123.68
    
    #BGR -> RGB
    x = x[:,:,::-1]
    x = np.clip(x,0,255).astype('unit8')
    return x

In [34]:
preprocess_image('content.jpg').shape

(1, 300, 300, 3)

In [35]:
# get tensor representations of our images
base_image = K.variable(preprocess_image('content.jpg'))
style_reference_image = K.variable(preprocess_image('style.jpg'))

In [40]:
base_image

In [41]:
combination_image = K.placeholder((1,img_width,img_height,3))

In [42]:
combination_image

<tf.Tensor 'Placeholder:0' shape=(1, 300, 300, 3) dtype=float32>

In [43]:
# combine the 3 images into a single tensor
input_tensor = K.concatenate([base_image, style_reference_image, combination_image], axis=0)

In [45]:
input_tensor.get_shape()

TensorShape([Dimension(3), Dimension(300), Dimension(300), Dimension(3)])

In [46]:
# build the vgg16 network with 3 images as input
# load pretrained image net weights

model = applications.VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)

In [47]:
# get outputs of each layer from the model

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [48]:
outputs_dict # lists out all the layers in the vgg16 network

{'block1_conv1': <tf.Tensor 'block1_conv1/Relu:0' shape=(3, 300, 300, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2/Relu:0' shape=(3, 300, 300, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool/MaxPool:0' shape=(3, 150, 150, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1/Relu:0' shape=(3, 150, 150, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2/Relu:0' shape=(3, 150, 150, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool/MaxPool:0' shape=(3, 75, 75, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1/Relu:0' shape=(3, 75, 75, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2/Relu:0' shape=(3, 75, 75, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3/Relu:0' shape=(3, 75, 75, 256) dtype=float32>,
 'block3_pool': <tf.Tensor 'block3_pool/MaxPool:0' shape=(3, 37, 37, 256) dtype=float32>,
 'block4_conv1': <tf.Tensor 'block4_conv1/Relu:0' shape=(3, 37, 37, 512) dtype=float32>,
 'block4_co

In [49]:
# define util functions for neural style loss

# gram matrix of image tensor
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2,0,1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [50]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels= 3
    size = img_width*img_height
    return K.sum(K.square(S-C))/(4.*(channels**2)*(size**2))

In [51]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [60]:
def total_variation_loss(x):
    a = K.square(x[:,:img_width-1,:img_height-1,:]- x[:,1:,:img_height-1,:])
    b = K.square(x[:,:img_width-1,:img_height-1,:]- x[:,:img_width-1,1:,:])
    return K.sum(K.pow(a+b, 1.25))
    

In [54]:
# combine loss into a single scalar

loss = K.variable(0.)
layer_features = outputs_dict['block4_conv2']

base_image_features = layer_features[0, :,:,:]
combination_features = layer_features[2,:,:,:]

loss += content_weight * content_loss(base_image_features, combination_features)

In [56]:
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

In [61]:
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1,:,:,:]
    combination_features = layer_features[2,:,:,:]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight/ len(feature_layers))*sl

loss += tv_weight * total_variation_loss(combination_image)

In [62]:
# get gradients of generated image wrt loss

grads = K.gradients(loss, combination_image)

In [64]:
outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

In [65]:
f_outputs = K.function([combination_image], outputs)

In [66]:
def eval_loss_and_grads(x):

    x = x.reshape((1,img_width, img_height, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [67]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [68]:
evaluator = Evaluator()

In [69]:
x = np.random.uniform(0, 255, (1, img_width, img_height, 3)) - 128.

In [ ]:
for i in range(10):
    print("Start of iteration", i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                    fprime=evaluator.grads, maxfun=20)
    print('Current loss value', min_val)
    #save generated image
    img = deprocess_image(x.copy())
    fname = 'result_at_iteration%d.png'%i
    imsave(fname, img)
    end_time = time.time()
    print("Image saved as", fname)
    print("End of iteration %d in %d secs" % (i,end_time - start_time))